# Chrome history restorer

If you lost your tabs,  this is a bit of improvement vs. going through the history and fishing out the links

Copy your history file to this directory

In [1]:
import sqlite3
import pandas as pd

In [ ]:
!ls

In [ ]:
conn = sqlite3.connect('History')
cursor = conn.cursor()

# Query to select URLs from the history
query = "SELECT url, last_visit_time FROM urls"
    
try:
    cursor.execute(query)
    data = cursor.fetchall()
    df = pd.DataFrame(data, columns=['URL', 'Last'])
    df['Last'] = pd.to_datetime((df['Last'] - 11644473600000000) / 1000000, unit='s', errors='coerce', utc=True)


except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    links = []
finally:
    conn.close()
    

In [ ]:
len(links)

In [ ]:
df.sort_values(by='Last', ascending=False)

In [ ]:
import math
import os
import pandas as pd
from urllib.parse import urlparse, parse_qs

def output_df_as_html_with_pagination(df, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Bootstrap CSS for styling
    bootstrap_css = '<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">'
    
    # Rename 'Last_Visit_Time' to 'Last'
    df.rename(columns={'Last_Visit_Time': 'Last'}, inplace=True)
    # Ensure 'Last' is datetime and handle NaT/NaN before converting to date
    df['Last'] = pd.to_datetime(df['Last'], errors='coerce').dt.date
    
    # Filter out rows where 'Last' is NaT/NaN (now None after dt.date conversion)
    df = df.dropna(subset=['Last'])
    
    # Adjust DataFrame order if needed
    cols = ['Last'] + [col for col in df if col != 'Last']
    df = df[cols]
    
    # Number of links per page
    links_per_page = 100
    num_pages = math.ceil(len(df) / links_per_page)
    
    # Function to format URL cell with ttl and uri if applicable
    def format_url_cell(url):
        parsed_url = urlparse(url)
        if parsed_url.netloc == 'noogafoofpebimajpfpamcfhoaifemoa':
            query_params = parse_qs(parsed_url.fragment)
            ttl = query_params.get('ttl', [''])[0]
            uri = query_params.get('uri', [''])[0][:15]
            return f'<a href="{url}" target="_blank">{url}</a><br>{ttl}<br>{uri}'
        else:
            return f'<a href="{url}" target="_blank">{url}</a>'
    
    for page in range(num_pages):
        # Calculate start and end indices for the current chunk
        start_idx = page * links_per_page
        end_idx = start_idx + links_per_page
        df_chunk = df.iloc[start_idx:end_idx]
        
        # Safely compute start and end dates for display
        start_date = df_chunk['Last'].min() if not df_chunk['Last'].isnull().all() else 'Unknown'
        end_date = df_chunk['Last'].max() if not df_chunk['Last'].isnull().all() else 'Unknown'
        
        # HTML structure with Bootstrap CSS
        html_start = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            {bootstrap_css}
            <title>Browser History - Page {page + 1}</title>
        </head>
        <body>
        <div class="container">
            <h2>Browser History</h2>
            <p>Displaying links from {start_date} to {end_date}</p>
        """
        html_end = """
        </div>
        </body>
        </html>
        """
        
        # Convert DataFrame chunk to HTML
        df_html = df_chunk.to_html(escape=False, formatters={
            'URL': format_url_cell,
            'Last': lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else 'Unknown'
        }, index=False, classes='table table-striped', columns=['Last', 'URL'])
        
        # Navigation links
        nav_links = '<div class="navigation">'
        if page > 0:
            nav_links += f'<a href="page_{page}.html">Previous</a>'
        if page < num_pages - 1:
            nav_links += f' <a href="page_{page + 2}.html">Next</a>'
        nav_links += '</div>'
        
        # Combine everything into a full HTML document
        full_html = f"{html_start}{nav_links}{df_html}{nav_links}{html_end}"
        
        # Write the HTML content to a file
        output_file_path = os.path.join(output_dir, f'page_{page + 1}.html')
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(full_html)

In [ ]:
sdf = df.sort_values(by='Last', ascending=False)
sdf = sdf[~sdf['URL'].str.startswith('https://observe')]
sdf = sdf[~sdf['URL'].str.startswith('https://admin.google')]
sdf = sdf[~sdf['URL'].str.startswith('https://mail.google.com')]

output_df_as_html_with_pagination(sdf, 'browser_history')

In [ ]:
sdf.describe()

In [ ]:
# Assuming 'df' is your original DataFrame and it contains a column named 'URL'
# Extract the first 15 characters of each URL to create a new DataFrame
prefix_df = sdf['URL'].str[:15].to_frame(name='URL_Prefix')

# Count occurrences of each prefix and get the top 100 most common
common_prefixes = prefix_df['URL_Prefix'].value_counts().head(100).to_frame(name='Count')

# Display the new DataFrame with common prefixes and their counts
print(common_prefixes)